# Deer Harvest - Total Harvest Review and Possible Associations with Chronic Wasting Disease

## Objectives
    - Compile data and identify state-wide trends on Total Deer Harvest
    - Map Year by Year Changes in Harvest Numbers
    - Map and Explore Chronic Wasting Disease relative to Deer Harvest
    

### Acquire Minnesota Basemap and import Libraries

In [ ]:
%matplotlib inline

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.simplefilter(action='ignore')


us_states = gpd.read_file('http://www2.census.gov/geo/tiger/GENZ2017/shp/cb_2017_us_state_20m.zip')
mn_state_boundary = us_states[us_states["STUSPS"].isin(["MN"])]


## Process all available yearly deer harvest data in Perpetual Permit Areas

The Minessota Geospatial Commons (https://gisdata.mn.gov/) has published Deer Harvest Data from 1993 to 2014.

One objective of this notebook is to compile all data available from 2009 onwards for Permit Areas that have not had their geographic boundaries altered (Perpetual Permit Areas). This will be an automated process and when years 2015 and later are released they can be downloaded and the notebook will automatically add them.

Note: Data before 2009 is not included in this process because Permit Area numbers, now designated as "kill blocks" were altered so we cannot automatically match them to post 2008 permit areas.

data source: https://gisdata.mn.gov/dataset/env-mn-deer-harvest

In [ ]:
# Get Deer Permit Areas Shapefile from file
# note: starting in 2009 permit areas are referred to as kill blocks in MN DNR gis data
deer_permit_areas = gpd.read_file('data\deer_permit_areas.json')

# Collect all deer harvest shapefiles from directory and append to single geodataframe
# As new yearly data is released we can re-run the process
folder = Path("data\deer_harvest_shp")
shapefiles = folder.glob("*.shp")

for shp in shapefiles:
    year = str(shp)[-8:-4]
    if int(year) >= 2009:
        dh_year = gpd.read_file(shp)
        # Reduce dataset to only columns of interest, rename them with associated year
        dh_year_reduce = dh_year[['KILLBLOCK','TOTALHARV']]
        dh_year_reduce.columns = ['KILLBLOCK',f'TOTALHARV_{year}']
        deer_permit_areas = deer_permit_areas.merge(dh_year_reduce, on='KILLBLOCK')
    else:
        pass
deer_permit_areas.sample(3)

## Overview of Permit Areas
In the following chart and maps we can observe which years had the greatest total harvest and which permit areas had t

In [ ]:
# General Statistics on total harvest data
deer_permit_areas[['TOTALHARV_2009','TOTALHARV_2010','TOTALHARV_2011','TOTALHARV_2012','TOTALHARV_2013','TOTALHARV_2014']].describe().plot.bar(figsize=(16, 10))

# View Maps at Relative Scale
Here we can take a look at maps for each year. The map colors are scaled for each individual year giving a sense of the full extent of total deer harvest values.

In [ ]:
# Get all total harvest columns names to plot a map for each year
years = list(deer_permit_areas.columns[4:])

# Calculate number of rows needed to plot all years
plot_rows = int(str(len(years)/2).rstrip('.0'))

# Create plot for every year available up to 9 years.
fig, axes = plt.subplots(figsize=(20, 30), ncols=2, nrows=plot_rows,
                        constrained_layout=True, 
                        sharex=True, sharey=True, 
                        subplot_kw=dict(aspect='equal'))

fig.suptitle('Total Deer Harvest in Perpetual Permit Areas (Relative Scales)', fontsize=35)

# Loop through up to 11 potential maps of Yearly Deer Harvest data
for idx, val in enumerate(years):
    if idx < 2:
        axes[0][idx].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[0][idx], column=years[idx], legend=True, cmap='coolwarm')
        deer_permit_areas.apply(lambda x: axes[0][idx].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 2 <= idx < 4:
        axes[1][idx-2].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[1][idx-2], column=years[idx], legend=True, cmap='coolwarm')
        deer_permit_areas.apply(lambda x: axes[1][idx-2].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 4 <= idx < 6:
        axes[2][idx-4].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[2][idx-4], column=years[idx], legend=True, cmap='coolwarm')
        deer_permit_areas.apply(lambda x: axes[2][idx-4].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 6 <= idx < 8:
        axes[2][idx-6].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[2][idx-6], column=years[idx], legend=True, cmap='coolwarm')
        deer_permit_areas.apply(lambda x: axes[2][idx-6].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 8 <= idx < 10:
        axes[3][idx-8].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[3][idx-8], column=years[idx], legend=True, cmap='coolwarm')
        deer_permit_areas.apply(lambda x: axes[3][idx-8].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 10 <= idx < 12:
        axes[3][idx-10].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[3][idx-10], column=years[idx], legend=True, cmap='coolwarm')
        deer_permit_areas.apply(lambda x: axes[3][idx-10].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
plt.show()

# View Maps at Common Scale
Now that we have a sense of the different total harvest values year to year it may be more helpful to view the maps at a common scale.

In [ ]:
# Grab years and set up plot
years = list(deer_permit_areas.columns[4:])
plot_rows = int(str(len(years)/2).rstrip('.0'))
fig, axes = plt.subplots(figsize=(20, 30), ncols=2, nrows=plot_rows,
                        constrained_layout=True, 
                        sharex=True, sharey=True, 
                        subplot_kw=dict(aspect='equal'))

fig.suptitle('Total Deer Harvest in Perpetual Permit Areas (Common Scale)', fontsize=35)

class_breaks = dict(bins=[0,1000,2000,3000,4000,5000,6000,7000,8000,9000])

# Loop through up to 11 potential maps of Yearly Deer Harvest data
for idx, val in enumerate(years):
    if idx < 2:
        axes[0][idx].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[0][idx], column=years[idx], legend=True,cmap='coolwarm',scheme="User_Defined",classification_kwds=class_breaks,legend_kwds={'loc': 'lower right','title': 'Total Deer Harvest','fontsize': 10})
        deer_permit_areas.apply(lambda x: axes[0][idx].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1) 
    elif 2 <= idx < 4:
        axes[1][idx-2].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[1][idx-2], column=years[idx], legend=True,cmap='coolwarm',scheme="User_Defined",classification_kwds=class_breaks,legend_kwds={'loc': 'lower right'})
        deer_permit_areas.apply(lambda x: axes[1][idx-2].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1) 
    elif 4 <= idx < 6:
        axes[2][idx-4].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[2][idx-4], column=years[idx], legend=False,cmap='coolwarm',scheme="User_Defined",classification_kwds=class_breaks,legend_kwds={'loc': 'lower right'})
        deer_permit_areas.apply(lambda x: axes[2][idx-4].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 6 <= idx < 8:
        axes[2][idx-6].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[2][idx-6], column=years[idx], legend=False,cmap='coolwarm',scheme="User_Defined",classification_kwds=class_breaks,legend_kwds={'loc': 'lower right'})
        deer_permit_areas.apply(lambda x: axes[2][idx-6].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 8 <= idx < 10:
        axes[3][idx-8].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[3][idx-8], column=years[idx], legend=False,cmap='coolwarm',scheme="User_Defined",classification_kwds=class_breaks,legend_kwds={'loc': 'lower right'})
        deer_permit_areas.apply(lambda x: axes[3][idx-8].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)
    elif 10 <= idx < 12:
        axes[3][idx-10].set_title(val[-4:], fontsize=25)
        deer_permit_areas.plot(ax=axes[3][idx-10], column=years[idx], legend=False,cmap='coolwarm',scheme="User_Defined",classification_kwds=class_breaks,legend_kwds={'loc': 'lower right'})
        deer_permit_areas.apply(lambda x: axes[3][idx-10].annotate(s=x.KILLBLOCK, xy=x.geometry.centroid.coords[0], ha='center'),axis=1)

plt.show()

# Permit Areas of Interest and Upcoming CWD Data
We can see from the maps that there is an area of high total harvest activity in the mid-north area of Minnesota. In particular Permit Area 241 has very high activity.

We will next be bringing in chronic wasting diease data and determing which Permit Area's CWD activity has been reported.

The CWD Data contains reported and confirmed cases of chronic wasting disease in Minnesota.

source: https://www.dnr.state.mn.us/cwdcheck/positive-deer-table.html

In [ ]:
# Read in chronic wasting diesease data
mn_cwd = gpd.read_file('data/cwd_positive_deer_data.csv')
mn_cwd['permit_area'] = pd.to_numeric(mn_cwd['permit_area'])
mn_cwd.head()


In [ ]:
# Join the CWD Data to the Deer Harvest Dataset we have been working with
mn_cwd.rename({'permit_area': 'KILLBLOCK'}, axis=1, inplace=True)
mn_cwd.info()
# deer_permit_areas.info()

deer_permit_areas = deer_permit_areas.merge(mn_cwd, on='KILLBLOCK', how='right')
deer_permit_areas.sample(3)